In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from PIL import ImageFile
from PIL import Image

ImageFile.LOAD_TRUNCATED_IMAGES = True

def safe_loader(path):
    try:
        with Image.open(path) as img:
            return img.convert('RGB')
    except:
        return None

class SafeImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if sample is None:
            # Skip or handle unreadable images
            return None, None
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

def safe_collate_fn(batch):
    # Exclude unreadable samples
    batch = [(x, y) for (x, y) in batch if x is not None and y is not None]
    if len(batch) == 0:
        return None, None
    return default_collate(batch)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
data_dir = "/kaggle/input/dataset/OC Dataset kaggle new"
train_data = SafeImageFolder(
    root=data_dir + '/train',
    transform=transform,
    loader=safe_loader
)
validation_data = SafeImageFolder(
    root=data_dir + '/valid',
    transform=transform,
    loader=safe_loader
)

trainloader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=safe_collate_fn)
valloader = DataLoader(validation_data, batch_size=32, shuffle=False, collate_fn=safe_collate_fn)

# Define model
model = models.resnet18(pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1),
    nn.Sigmoid()
)
model = model.to(device)

# Define loss and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
best_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        
        # Binary prediction threshold at 0.5
        preds = (outputs >= 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_data)
    epoch_acc = correct / total if total > 0 else 0
    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}")
    
    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            preds = (outputs >= 0.5).float()
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= len(validation_data)
    val_acc = val_correct / val_total if val_total > 0 else 0
    print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")

print("Training complete.")


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10, Training Loss: 0.5951, Training Accuracy: 0.8106
Epoch 1/10, Validation Loss: 0.6582, Validation Accuracy: 0.6633
Epoch 2/10, Training Loss: 0.5711, Training Accuracy: 0.8606
Epoch 2/10, Validation Loss: 0.6215, Validation Accuracy: 0.8112
Epoch 3/10, Training Loss: 0.5463, Training Accuracy: 0.9136
Epoch 3/10, Validation Loss: 0.7059, Validation Accuracy: 0.6735
Epoch 4/10, Training Loss: 0.5529, Training Accuracy: 0.8955
Epoch 4/10, Validation Loss: 0.5765, Validation Accuracy: 0.8520
Epoch 5/10, Training Loss: 0.5509, Training Accuracy: 0.9000
Epoch 5/10, Validation Loss: 0.6118, Validation Accuracy: 0.8265
Epoch 6/10, Training Loss: 0.5385, Training Accuracy: 0.9318
Epoch 6/10, Validation Loss: 0.6220, Validation Accuracy: 0.6939
Epoch 7/10, Training Loss: 0.5384, Training Accuracy: 0.9258
Epoch 7/10, Validation Loss: 0.5905, Validation Accuracy: 0.8316
Epoch 8/10, Training Loss: 0.5408, Training Accuracy: 0.9258
Epoch 8/10, Validation Loss: 0.6303, Validation Accuracy:

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from PIL import ImageFile
from PIL import Image

ImageFile.LOAD_TRUNCATED_IMAGES = True

def safe_loader(path):
    try:
        with Image.open(path) as img:
            return img.convert('RGB')
    except:
        return None

class SafeImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)
        if sample is None:
            # Skip or handle unreadable images
            return None, None
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

def safe_collate_fn(batch):
    # Exclude unreadable samples
    batch = [(x, y) for (x, y) in batch if x is not None and y is not None]
    if len(batch) == 0:
        return None, None
    return default_collate(batch)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load dataset
data_dir = "/kaggle/input/dataset/OC Dataset kaggle new"
train_data = SafeImageFolder(
    root=data_dir + '/train',
    transform=transform,
    loader=safe_loader
)
validation_data = SafeImageFolder(
    root=data_dir + '/valid',
    transform=transform,
    loader=safe_loader
)

trainloader = DataLoader(train_data, batch_size=32, shuffle=True, collate_fn=safe_collate_fn)
valloader = DataLoader(validation_data, batch_size=32, shuffle=False, collate_fn=safe_collate_fn)

# Define model with 2 outputs and no sigmoid
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes
model = model.to(device)

# Use CrossEntropyLoss for two-class output
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
best_val_loss = float('inf')

for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for inputs, labels in trainloader:
        # Labels: shape [batch_size], values 0 or 1
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)     # shape [batch_size, 2]
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = outputs.max(dim=1)        # get predicted class
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_data)
    epoch_acc = correct / total if total > 0 else 0
    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

    # Validation loop
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in valloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            
            _, preds = outputs.max(dim=1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= len(validation_data)
    val_acc = val_correct / val_total if val_total > 0 else 0
    print(f"Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model2.pth")
        
print("Training complete.")


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 174MB/s] 
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/10, Training Loss: 0.5018, Accuracy: 0.8091
Epoch 1/10, Validation Loss: 9.3561, Validation Accuracy: 0.5663
Epoch 2/10, Training Loss: 0.3397, Accuracy: 0.8576
Epoch 2/10, Validation Loss: 0.9228, Validation Accuracy: 0.6480
Epoch 3/10, Training Loss: 0.1394, Accuracy: 0.9500
Epoch 3/10, Validation Loss: 0.6016, Validation Accuracy: 0.7806
Epoch 4/10, Training Loss: 0.0492, Accuracy: 0.9848
Epoch 4/10, Validation Loss: 0.4055, Validation Accuracy: 0.8469
Epoch 5/10, Training Loss: 0.1001, Accuracy: 0.9636
Epoch 5/10, Validation Loss: 0.9772, Validation Accuracy: 0.8163
Epoch 6/10, Training Loss: 0.1393, Accuracy: 0.9485
Epoch 6/10, Validation Loss: 0.6925, Validation Accuracy: 0.8520
Epoch 7/10, Training Loss: 0.0951, Accuracy: 0.9758
Epoch 7/10, Validation Loss: 0.4507, Validation Accuracy: 0.8571
Epoch 8/10, Training Loss: 0.0465, Accuracy: 0.9833
Epoch 8/10, Validation Loss: 1.1749, Validation Accuracy: 0.7806
Epoch 9/10, Training Loss: 0.0353, Accuracy: 0.9894
Epoch 9/10, 